# Imports and Hyperparameters

In [1]:
import pickle
import random
import numpy as np
import pandas as pd
import os
from datetime import datetime
import json
from collections import defaultdict
import plotly.express as px
import plotly.graph_objects as go
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from SGN.model import SGN
from SGN.data import NTUDataLoaders, AverageMeter
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
import mlflow

In [2]:
# Hyper Parameters
one_dimension_conv = False
ntu_120 = False
seperate_train_test = False
T = 75
dataset = 'NTU'
device = torch.device('cuda:0')
seg = 20
lr = 5e-5
if ntu_120:
    utility_classes = 120
    privacy_classes = 106
else:
    utility_classes = 60
    privacy_classes = 40
validation_acc_freq = 10 #-1 to disable
encoded_channels = (512, 32)
batch_size = 32
workers=0
cross_samples_train = 10000
cross_samples_test = 5000

# Data

## Import and Organize

X = (frames, joints, pos + orientation)
    
    (frames, 25, 7)

In [3]:
# load data
with open('ntu/X.pkl', 'rb') as f:
    X = pickle.load(f)

# pad/trim data to T frames and convert to tensor
for file, value in X.items():
    num_frames = value.shape[0]

    # Pad or trim
    if num_frames < T:
        padding = np.repeat(value[-1][np.newaxis, :, :], T - num_frames, axis=0)
        value = np.concatenate((value, padding), axis=0)
    elif num_frames > T:
        value = value[:T]
    
    # Convert to tensor and store back
    X[file] = torch.from_numpy(value).float()

if not ntu_120:
    to_rem = []
    for file in X.keys():
        if int(file.split('A')[1][:3]) > 60:
            to_rem.append(file)
    print(len(to_rem))
    for file in to_rem:
        del X[file]

40236


## Visualization Function

In [4]:
def render_frame(d):
    reshaped_data = d.reshape(-1, 3)
    x = reshaped_data[:, 0]
    y = reshaped_data[:, 1]
    z = reshaped_data[:, 2]

    df = pd.DataFrame({'x': x, 'y': y, 'z': z})

    fig = px.scatter_3d(df, x='x', y='y', z='z', color=np.linspace(1, 25, len(x)),
                        color_continuous_scale='Rainbow', title='Interactive 3D Scatter Plot')

    fig.update_traces(marker=dict(size=2))

    cons = [[0, 1], [1, 20], [20, 2], [2, 3], [20, 8], [8, 9], [9, 10], [10, 11], [11, 23], [11, 24], [20, 4], [4, 5], [5, 6], [6, 7], [7, 21], [7, 22], [0, 16], [16, 17], [17, 18], [18, 19], [0, 12], [12, 13], [13, 14], [14, 15]]

    for con in cons:
        lx = [x[con[0]], x[con[1]]]
        ly = [y[con[0]], y[con[1]]]
        lz = [z[con[0]], z[con[1]]]
        fig.add_trace(go.Scatter3d(x=lx, y=ly, z=lz, mode='lines', line=dict(color='black', width=2)))

    fig.show()

def render_video(d):
    cons = [[0, 1], [1, 20], [20, 2], [2, 3], [20, 8], [8, 9], [9, 10], [10, 11], [11, 23], [11, 24], [20, 4], [4, 5], [5, 6], [6, 7], [7, 21], [7, 22], [0, 16], [16, 17], [17, 18], [18, 19], [0, 12], [12, 13], [13, 14], [14, 15]]

    frame_data = d[0].reshape(-1, 3)
    x = frame_data[:, 0]
    y = frame_data[:, 1]
    z = frame_data[:, 2]

    scatter = go.Scatter3d(x=x, y=y, z=z, mode='markers',
                        marker=dict(size=2, color=np.linspace(1, 25, 25), colorscale='Rainbow'))

    traces = [scatter]

    for con in cons:
        lx = [x[con[0]], x[con[1]]]
        ly = [y[con[0]], y[con[1]]]
        lz = [z[con[0]], z[con[1]]]
        line_trace = go.Scatter3d(x=lx, y=ly, z=lz, mode='lines', line=dict(color='black', width=2))
        traces.append(line_trace)

    layout = go.Layout(updatemenus=[dict(type='buttons', showactive=False,
                                        buttons=[dict(label='Play',
                                                    method='animate',
                                                    args=[None, dict(frame=dict(duration=100, redraw=True), fromcurrent=True)])])],
                    sliders=[dict(steps=[])],
                    title="Animated 3D Scatter Plot with Connections"
            )

    fig = go.Figure(data=traces, layout=layout)

    frame_list = []

    for i in range(d.shape[0]):
        frame_data = d[i].reshape(-1, 3)
        x, y, z = frame_data[:, 0], frame_data[:, 1], frame_data[:, 2]
    
        fig.data[0].x = x
        fig.data[0].y = y
        fig.data[0].z = z
        
        center_x, center_y, center_z = x[0], y[0], z[0]
        axis_bound = 2

        fig.update_layout(scene=dict(
            xaxis=dict(range=[center_x - axis_bound, center_x + axis_bound]),
            yaxis=dict(range=[center_y - axis_bound, center_y + axis_bound]),
            zaxis=dict(range=[center_z - axis_bound, center_z + axis_bound])
        ))

        frame_traces = []

        frame_scatter = go.Scatter3d(x=x, y=y, z=z, mode='markers',
                                    marker=dict(size=2, color=np.linspace(1, 25, 25), colorscale='Rainbow'))
        frame_traces.append(frame_scatter)

        for con in cons:
            lx = [x[con[0]], x[con[1]]]
            ly = [y[con[0]], y[con[1]]]
            lz = [z[con[0]], z[con[1]]]
            line_trace = go.Scatter3d(x=lx, y=ly, z=lz, mode='lines', line=dict(color='black', width=2))
            frame_traces.append(line_trace)

        frame = go.Frame(data=frame_traces, name=f'Frame {i}')
        frame_list.append(frame)

    fig.frames = frame_list

    fig.show()

## Cross Data Logic

In [45]:
def parse_file_name(file_name):
    """Parses the filename into a dictionary of parts."""
    parts = file_name.split('C')
    S = parts[0][1:]
    parts = parts[1].split('P')
    C = parts[0]
    parts = parts[1].split('R')
    P = parts[0]
    parts = parts[1].split('A')
    R = parts[0]
    parts = parts[1].split('.')
    A = parts[0]
    return {'S': S, 'C': C, 'P': P, 'R': R, 'A': A}

def organize_data(data):
    organized_data = defaultdict(list)
    for file_name, content in data.items():
        parts = parse_file_name(file_name)
        if not ntu_120: # NTU 60
            if int(parts['A']) > 60:
                continue
        organized_data[(parts['C'], parts['R'])].append((parts['P'], parts['A'], content))
    return organized_data

def sample_data(organized_data):
    # Pick a random (C, R) pair
    cr_pair = random.choice(list(organized_data.keys()))

    # Get all (P, A, content) tuples for this (C, R) pair
    pa_list = organized_data[cr_pair]

    # Pick 2 unique P values and 2 unique A values
    random.shuffle(pa_list)
    unique_p = set()
    unique_a = set()
    for p, a, content in pa_list:
        if len(unique_p) < 2:
            unique_p.add(p)
        if len(unique_a) < 2:
            unique_a.add(a)
        if len(unique_p) == 2 and len(unique_a) == 2:
            break

    if len(unique_p) < 2 or len(unique_a) < 2:
        raise Exception(f'Not enough unique P or A values for (C, R) pair {cr_pair}')

    # Form all four (P, A) pairs and get the corresponding content
    sampled_data = []
    for p in unique_p:
        for a in unique_a:
            for pa_content in pa_list:
                if pa_content[0] == p and pa_content[1] == a:
                    sampled_data.append(pa_content)
                    break

    return sampled_data

def gen_samples(samples, data):
    d = []
    unique_samples = set()  # Set to track unique samples
    for _ in range(samples):
        failed = 0
        while True:
            d_ = sample_data(data)
            d_tuple = tuple(tuple(x) for x in d_)
            if d_tuple not in unique_samples and len(d_tuple) == 4:
                unique_samples.add(d_tuple)  # Add the unique sample to the set
                d.append(d_)  # Add the unique sample to the dataset
                break
            failed += 1
            if failed > 100:
                print('failed to sample data')
                break
    return np.array(d)


organized_data = organize_data(X)

## Reconstruction Sampling

In [6]:
def sample_rec_data(X):
    # Remove NTU 120 if needed
    if not ntu_120:
        X = {k: v for k, v in X.items() if int(parse_file_name(k)['A']) <= 60}

    # Split data into train and test
    X_train_keys, X_test_keys = train_test_split(list(X.keys()), test_size=0.2, random_state=42)
    
    # Create train and test sets
    X_train = np.zeros((len(X_train_keys), T, 25, 7))
    X_test = np.zeros((len(X_test_keys), T, 25, 7))
    for i, key in enumerate(X_train_keys):
        X_train[i] = X[key]
    for i, key in enumerate(X_test_keys):
        X_test[i] = X[key]

    # Get actor and action names
    train_actors = [parse_file_name(key)['P'] for key in X_train_keys]
    test_actors = [parse_file_name(key)['P'] for key in X_test_keys]
    train_actions = [parse_file_name(key)['A'] for key in X_train_keys]
    test_actions = [parse_file_name(key)['A'] for key in X_test_keys]
    
    return X_train, X_test, train_actors, train_actions, test_actors, test_actions

## Sample Data

In [50]:
class Cross_Data(Dataset):
    def __init__(self, sampled_data):
        self.data = sampled_data # the tuple is actor, action, frames
        self.x1 = sampled_data[:, 0, 2] # P1, A1
        self.x2 = sampled_data[:, 3, 2] # P2, A2
        self.y1 = sampled_data[:, 1, 2] # P1, A2
        self.y2 = sampled_data[:, 2, 2] # P2, A1

    def __getitem__(self, index): # data, actors, actions
        return  self.x1[index][:, :, 0:3], self.x1[index][:, :, 3:7],\
                self.x2[index][:, :, 0:3], self.x2[index][:, :, 3:7],\
                self.y1[index][:, :, 0:3], self.y1[index][:, :, 3:7],\
                self.y2[index][:, :, 0:3], self.y2[index][:, :, 3:7],\
                [float(self.data[index][0][0]), float(self.data[index][3][0])], [float(self.data[index][0][1]), float(self.data[index][1][1])]
    
    def __len__(self):
        return len(self.data)

class Rec_Data(Dataset):
    def __init__(self, X, Actor, Action):
        self.X = X
        self.Actor = Actor
        self.Action = Action
    
    def __getitem__(self, index):
        return self.X[index], float(self.Actor[index]), float(self.Action[index])
    
    def __len__(self):
        return len(self.X)

# Cross Data
train_data = gen_samples(cross_samples_train, organized_data)
if seperate_train_test: val_data = gen_samples(cross_samples_test, organized_data)
else: val_data = train_data
train_dataset = Cross_Data(train_data)
val_dataset = Cross_Data(val_data)
train_dl = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dl = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
# Rec Data
rec_train_data, rec_val_data, t_actors, t_actions, v_actors, v_actions = sample_rec_data(X)
rec_train_dataset = Rec_Data(rec_train_data, t_actors, t_actions)
rec_val_dataset = Rec_Data(rec_val_data, v_actors, v_actions)
rec_train_dl = DataLoader(rec_train_dataset, batch_size=batch_size, shuffle=True)
rec_val_dl = DataLoader(rec_val_dataset, batch_size=batch_size, shuffle=True)

C:\Users\Carrt\AppData\Local\Temp\ipykernel_2700\283437463.py:74: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(d)


# Model

## Adversary

In [51]:
# Input is size of latent space
class Adversary_Emb(nn.Module):
    def __init__(self, num_classes):
        super(Adversary_Emb, self).__init__()
        self.channels = 64
        self.conv = nn.ConvTranspose1d(encoded_channels[0], self.channels, 3, 1, 1)
        self.ref = nn.ReflectionPad1d(3)
        self.pool = nn.MaxPool1d(2, 2)
        self.fc1 = nn.Linear(64 * (encoded_channels[1] + 6) // 2, 256)
        self.fc2 = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.ref(x)
        x = self.conv(x)
        x = self.pool(x)
        x = x.view(x.shape[0], -1)
        x = F.relu(self.fc1(x))
        x = F.softmax(self.fc2(x), dim=1)
        return x
    
class Discriminator(nn.Module): # 1 = real, 0 = fake
    def __init__(self):
        super(Discriminator, self).__init__()

        self.enc1 = nn.Conv1d(in_channels=T, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.enc2 = nn.Conv1d(in_channels=64, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.enc3 = nn.Conv1d(in_channels=32, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.enc4 = nn.Conv1d(in_channels=16, out_channels=8, kernel_size=3, stride=1, padding=1)
        self.ref1 = nn.ReflectionPad1d(3)
        self.ref2 = nn.ReflectionPad1d(3)
        self.ref3 = nn.ReflectionPad1d(3)
        self.ref4 = nn.ReflectionPad1d(3)
        self.fc1 = nn.Linear(80, 32)
        self.fc2 = nn.Linear(32, 1)

        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)

        self.acti = nn.LeakyReLU(0.2)

    def forward(self, x):
        x = self.ref1(x)
        x = self.acti(self.enc1(x))
        x = self.pool(x)
        
        x = self.ref2(x)
        x = self.acti(self.enc2(x))
        x = self.pool(x)
        
        x = self.ref3(x)
        x = self.acti(self.enc3(x))
        x = self.pool(x)

        x = self.ref4(x)
        x = self.acti(self.enc4(x))
        x = self.pool(x)

        #flatten
        x = x.view(x.shape[0], -1)
        x = F.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))

        return x

## Motion Retargeting

In [52]:
class Encoder1D(nn.Module):
    def __init__(self):
        super(Encoder1D, self).__init__()

        self.enc1 = nn.Conv1d(in_channels=T, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.enc2 = nn.Conv1d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.enc3 = nn.Conv1d(in_channels=256, out_channels=512, kernel_size=3, stride=1, padding=1)
        self.enc4 = nn.Conv1d(in_channels=512, out_channels=encoded_channels[0], kernel_size=3, stride=1, padding=1)
        self.ref1 = nn.ReflectionPad1d(3)
        self.ref2 = nn.ReflectionPad1d(3)
        self.ref3 = nn.ReflectionPad1d(3)
        self.ref4 = nn.ReflectionPad1d(3)

        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)

        self.acti = nn.LeakyReLU(0.2)

        self.global_avg_pool = nn.AdaptiveAvgPool1d(1)
        self.fc1 = nn.Linear(encoded_channels[0], encoded_channels[0] * encoded_channels[1])

    def forward(self, x):
        x = self.ref1(x)
        x = self.acti(self.enc1(x))
        x = self.pool(x)
        
        x = self.ref2(x)
        x = self.acti(self.enc2(x))
        x = self.pool(x)
        
        x = self.ref3(x)
        x = self.acti(self.enc3(x))
        x = self.pool(x)

        x = self.ref4(x)
        x = self.acti(self.enc4(x))
        x = self.pool(x)

        x = self.global_avg_pool(x)
        x = x.squeeze(-1) 
        x = self.fc1(x)
        x = x.view(-1, *encoded_channels)

        return x

class Decoder1D(nn.Module):
    def __init__(self):
        super(Decoder1D, self).__init__()

        self.dec1 = nn.ConvTranspose1d(in_channels=encoded_channels[0]*2, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.dec2 = nn.ConvTranspose1d(in_channels=256, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.dec3 = nn.ConvTranspose1d(in_channels=128, out_channels=96, kernel_size=3, stride=1, padding=1)
        self.dec4 = nn.ConvTranspose1d(in_channels=96, out_channels=T, kernel_size=3, stride=1, padding=1)

        self.ref1 = nn.ReflectionPad1d(3)
        self.ref2 = nn.ReflectionPad1d(3)
        self.ref3 = nn.ReflectionPad1d(3)
        self.ref4 = nn.ReflectionPad1d(3)
 
        self.up = nn.Upsample(scale_factor=2, mode='nearest')
        self.up75 = nn.Upsample(size=T, mode='nearest') 

        self.acti = nn.LeakyReLU(0.2)

    def forward(self, x):
        x = self.ref1(x)
        x = self.acti(self.dec1(x))
        x = self.up(x)

        x = self.ref2(x)
        x = self.acti(self.dec2(x))
        x = self.up(x)

        x = self.ref3(x)
        x = self.acti(self.dec3(x))
        x = self.up(x)

        x = self.ref4(x)
        x = self.acti(self.dec4(x))
        x = self.up75(x)
        return x
    
class Encoder2D(nn.Module):
    def __init__(self):
        super(Encoder2D, self).__init__()

        self.enc1 = nn.Conv2d(in_channels=T, out_channels=12, kernel_size=(3,3), stride=1, padding=1)
        self.enc2 = nn.Conv2d(in_channels=12, out_channels=24, kernel_size=(3,3), stride=1, padding=1)
        self.enc3 = nn.Conv2d(in_channels=24, out_channels=32, kernel_size=(3,3), stride=1, padding=1)
        self.enc4 = nn.Conv2d(in_channels=32, out_channels=encoded_channels[0], kernel_size=(3,3), stride=1, padding=1)

        self.ref = nn.ReflectionPad2d(1)
        self.pool = nn.MaxPool2d(kernel_size=(2,2), stride=2)
        self.acti = nn.LeakyReLU(0.2)

        self.global_avg_pool = nn.AdaptiveAvgPool2d((1,1))
        self.fc1 = nn.Linear(encoded_channels[0], encoded_channels[0] * encoded_channels[1])

    def forward(self, x):
        x = self.ref(x)
        x = self.acti(self.enc1(x))
        x = self.pool(x)

        x = self.ref(x)
        x = self.acti(self.enc2(x))
        x = self.pool(x)
        
        x = self.ref(x)
        x = self.acti(self.enc3(x))
        x = self.pool(x)

        x = self.ref(x)
        x = self.acti(self.enc4(x))
        x = self.pool(x)

        x = self.global_avg_pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = x.view(-1, *encoded_channels)

        return x

class Decoder2D(nn.Module):
    def __init__(self):
        super(Decoder2D, self).__init__()

        self.dec1 = nn.ConvTranspose2d(in_channels=encoded_channels[0]*2, out_channels=256, kernel_size=(3,3), stride=1, padding=1)
        self.dec2 = nn.ConvTranspose2d(in_channels=256, out_channels=128, kernel_size=(3,3), stride=1, padding=1)
        self.dec3 = nn.ConvTranspose2d(in_channels=128, out_channels=96, kernel_size=(3,3), stride=1, padding=1)
        self.dec4 = nn.ConvTranspose2d(in_channels=96, out_channels=T, kernel_size=(3,3), stride=1, padding=1)

        self.ref = nn.ReflectionPad2d(3)
        self.up = nn.Upsample(scale_factor=2, mode='nearest')
        self.up75 = nn.Upsample(size=T, mode='nearest') 
        self.acti = nn.LeakyReLU(0.2)

    def forward(self, x):
        x = self.ref(x)
        x = self.acti(self.dec1(x))
        x = self.up(x)

        x = self.ref(x)
        x = self.acti(self.dec2(x))
        x = self.up(x)

        x = self.ref(x)
        x = self.acti(self.dec3(x))
        x = self.up(x)

        x = self.ref(x)
        x = self.acti(self.dec4(x))
        x = self.up75(x)
        
        return x

class AutoEncoder(nn.Module):
    def __init__(self, adv_lr=1e-3, use_adv=True):
        super(AutoEncoder, self).__init__()

        # AutoEncoder Models
        if one_dimension_conv:
            self.static_encoder = Encoder1D()
            self.dynamic_encoder = Encoder1D()
            self.decoder = Decoder1D()
        else:
            self.static_encoder = Encoder2D()
            self.dynamic_encoder = Encoder2D()
            self.decoder = Decoder1D()

        # Adversarial Models
        self.use_adv = use_adv
        if use_adv:
            self.priv_adv = Adversary_Emb(privacy_classes)
            self.util_adv = Adversary_Emb(utility_classes)
            self.discriminator = Discriminator()

            self.priv_optim = torch.optim.Adam(self.priv_adv.parameters(), lr=adv_lr)
            self.util_optim = torch.optim.Adam(self.util_adv.parameters(), lr=adv_lr)
            self.discriminator_optim = torch.optim.Adam(self.discriminator.parameters(), lr=adv_lr)

            # Freeze Adversarial Models
            self.priv_adv.eval()
            self.util_adv.eval()
            self.discriminator.eval()

        # Loss Functions
        self.triplet_loss = nn.TripletMarginLoss()
        self.bce_loss = nn.BCELoss()
        
        # Info for loss functions
        self.end_effectors = torch.tensor([19, 15, 23, 24, 21, 22, 3]).to(device) * 3
        self.chain_lengths = torch.tensor([5, 5, 8, 8, 8, 8, 5]).to(device)

        # Lambdas for discounted loss
        self.lambda_rec = 2
        self.lambda_cross = 0.1
        self.lambda_ee = 1
        self.lambda_trip = 0.1
        self.lambda_latent = 5
        self.lambda_adv_util = 0.25
        self.lambda_adv_priv = 0.25
        self.lambda_adv_disc = 0.25

        # Loss Toggles
        self.use_rec_loss = True
        self.use_cross_loss = True
        self.use_ee_loss = True 
        self.use_trip_loss = False # Remove comments when re-enabling
        self.use_latent_consistency = True

    def get_loss_params(self):
        return {
            'lambda_rec': self.lambda_rec,
            'lambda_cross': self.lambda_cross,
            'lambda_ee': self.lambda_ee,
            'lambda_trip': self.lambda_trip,
            'lambda_latent': self.lambda_latent,
            'lambda_adv_util': self.lambda_adv_util,
            'lambda_adv_priv': self.lambda_adv_priv,
            'lambda_adv_disc': self.lambda_adv_disc,
            'use_rec_loss': self.use_rec_loss,
            'use_cross_loss': self.use_cross_loss,
            'use_ee_loss': self.use_ee_loss,
            'use_trip_loss': self.use_trip_loss,
            'use_latent_consistency': self.use_latent_consistency
        }

    def cross(self, x1, x1_rot, x2, x2_rot):
        d1 = self.dynamic_encoder(x1_rot)
        d2 = self.dynamic_encoder(x2_rot)
        s1 = self.static_encoder(x1)
        s2 = self.static_encoder(x2)
        
        x1_hat = self.decoder(torch.cat((d1, s1), dim=1))
        x2_hat = self.decoder(torch.cat((d2, s2), dim=1))
        y1_hat = self.decoder(torch.cat((d1, s2), dim=1))
        y2_hat = self.decoder(torch.cat((d2, s1), dim=1))

        return x1_hat, x2_hat, y1_hat, y2_hat
    
    def eval(self, x1_rot, x2):
        dynamic = self.dynamic_encoder(x1_rot)
        static = self.static_encoder(x2)
        return self.decoder(torch.cat((dynamic, static), dim=1))

    def rec_loss(self, x, x_rot):
        d = self.dynamic_encoder(x_rot)
        s = self.static_encoder(x)
        x_hat = self.decoder(torch.cat((d, s), dim=1))
        if not one_dimension_conv:
            x_ = x.reshape(x.size(0), T, -1)
        return self.reconstruction_loss(x_, x_hat)
    
    def loss_paired(self, x1, x1_rot, x2, x2_rot, y1, y1_rot, y2, y2_rot, actors, actions, cross = True, reconstruction = True, emb_adv = True, discrim_adv = True, verbose = False):
        d1 = self.dynamic_encoder(x1_rot) # A1
        d2 = self.dynamic_encoder(x2_rot) # A2
        s1 = self.static_encoder(x1) # P1
        s2 = self.static_encoder(x2) # P2

        x1_hat = self.decoder(torch.cat((d1, s1), dim=1)) # P1, A1
        x2_hat = self.decoder(torch.cat((d2, s2), dim=1)) # P2, A2
        y1_hat = self.decoder(torch.cat((d1, s2), dim=1)) # P2, A1
        y2_hat = self.decoder(torch.cat((d2, s1), dim=1)) # P1, A2

        d12 = self.dynamic_encoder(y1_rot) # A1
        d21 = self.dynamic_encoder(y2_rot) # A2
        s12 = self.static_encoder(y1) # P2
        s21 = self.static_encoder(y2) # P1

        x1_hat_ = self.decoder(torch.cat((d12, s21), dim=1)) # P1, A1
        x2_hat_ = self.decoder(torch.cat((d21, s12), dim=1)) # P2, A2
        y1_hat_ = self.decoder(torch.cat((d12, s12), dim=1)) # P2, A1
        y2_hat_ = self.decoder(torch.cat((d21, s21), dim=1)) # P1, A2

        # flatten data if 2D
        if not one_dimension_conv:
            x1 = x1.view(x1.size(0), T, -1)
            x2 = x2.view(x2.size(0), T, -1)
            y1 = y1.view(y1.size(0), T, -1)
            y2 = y2.view(y2.size(0), T, -1)
        
        # initialize all losses to 0 tensor
        rec_loss = torch.zeros(1).to(device)
        cross_loss = torch.zeros(1).to(device)
        end_effector_loss = torch.zeros(1).to(device)
        triplet_loss = torch.zeros(1).to(device)
        latent_consistency_loss = torch.zeros(1).to(device)
        privacy_loss = torch.zeros(1).to(device)
        privacy_loss_dyn = torch.zeros(1).to(device)
        privacy_loss_stat = torch.zeros(1).to(device)
        privacy_acc_dyn = torch.zeros(1).to(device)
        privacy_acc_stat = torch.zeros(1).to(device)
        utility_loss = torch.zeros(1).to(device)
        utility_loss_dyn = torch.zeros(1).to(device)
        utility_loss_stat = torch.zeros(1).to(device)
        utility_acc_dyn = torch.zeros(1).to(device)
        utility_acc_stat = torch.zeros(1).to(device)
        discriminator_loss = torch.zeros(1).to(device)
        discriminator_acc = torch.zeros(1).to(device)
                        
        # reconstruction loss
        if self.use_rec_loss and reconstruction:
            rec_loss = self.reconstruction_loss(x1, x1_hat) + self.reconstruction_loss(x2, x2_hat) + self.reconstruction_loss(y1, y1_hat_) + self.reconstruction_loss(y2, y2_hat_)
            if verbose: print('Reconstruction Loss: ', rec_loss.item())
        
        # cross reconstruction loss
        if self.use_cross_loss and cross:
            cross_loss = self.cross_loss(y1, y2, y1_hat, y2_hat) + self.cross_loss(x1, x2, x1_hat_, x2_hat_)
            if verbose: print('Cross Reconstruction Loss: ', cross_loss.item())
        
        # end effector loss
        if self.use_ee_loss:
            if reconstruction:
                end_effector_loss += self.end_effector_loss(x1_hat, x1) + self.end_effector_loss(x2_hat, x2)
            if cross:
                end_effector_loss += self.end_effector_loss(y1_hat, y1) + self.end_effector_loss(y2_hat, y2)
            if verbose: print('End Effector Loss: ', end_effector_loss.item())

        # triplet loss
        if self.use_trip_loss:
            triplet_loss = self.triplet_loss(d12, d1, d2) + self.triplet_loss(d21, d2, d1) + self.triplet_loss(s12, s1, s2) + self.triplet_loss(s21, s2, s1) 
            if verbose: print('Triplet Loss: ', triplet_loss.item())

        # latent consistency loss
        if self.use_latent_consistency:
            latent_consistency_loss = self.latent_consistency_loss(d1, d12) + self.latent_consistency_loss(d2, d21) + self.latent_consistency_loss(s1, s21) + self.latent_consistency_loss(s2, s12)
            if verbose: print('Latent Consistency Loss: ', latent_consistency_loss.item())

        # adversarial loss
        if self.use_adv and emb_adv:
            # latent privacy loss (adversarial)
            adv_priv_y1, adv_priv_y2 = actors[0] - 1, actors[1] - 1
            adv_priv_y1, adv_priv_y2 = torch.eye(privacy_classes)[adv_priv_y1.long()].to(device), torch.eye(privacy_classes)[adv_priv_y2.long()].to(device)
            privacy_loss_dyn = -(self.adv_loss(self.priv_adv, d1, adv_priv_y1) + self.adv_loss(self.priv_adv, d2, adv_priv_y2))
            privacy_loss_stat = (self.adv_loss(self.priv_adv, s1, adv_priv_y1) + self.adv_loss(self.priv_adv, s2, adv_priv_y2))
            privacy_loss = privacy_loss_dyn + privacy_loss_stat
            if verbose: print('Privacy Loss Dynamic: ', privacy_loss_dyn.item(), '\tPrivacy Loss Static: ', privacy_loss_stat.item(), '\tPrivacy Loss: ', privacy_loss.item())

            privacy_acc_dyn = (self.adv_accuracy(self.priv_adv, d1, adv_priv_y1) + self.adv_accuracy(self.priv_adv, d2, adv_priv_y2)) / 2
            privacy_acc_stat = (self.adv_accuracy(self.priv_adv, s1, adv_priv_y1) + self.adv_accuracy(self.priv_adv, s2, adv_priv_y2)) / 2
            if verbose: print('Privacy Accuracy Dynamic: ', privacy_acc_dyn.item(), '\tPrivacy Accuracy Static: ', privacy_acc_stat.item())
            
            # latent utility loss (adversarial)
            adv_util_y1, adv_util_y2 = actions[0] - 1, actions[1] - 1
            adv_util_y1, adv_util_y2 = torch.eye(utility_classes)[adv_util_y1.long()].to(device), torch.eye(utility_classes)[adv_util_y2.long()].to(device)
            utility_loss_dyn = (self.adv_loss(self.util_adv, d1, adv_util_y1) + self.adv_loss(self.util_adv, d2, adv_util_y2))
            utility_loss_stat = -(self.adv_loss(self.util_adv, s1, adv_util_y1) + self.adv_loss(self.util_adv, s2, adv_util_y2))
            utility_loss = utility_loss_dyn + utility_loss_stat
            if verbose: print('Utility Loss Dynamic: ', utility_loss_dyn.item(), '\tUtility Loss Static: ', utility_loss_stat.item(), '\tUtility Loss: ', utility_loss.item())

            utility_acc_dyn = (self.adv_accuracy(self.util_adv, d1, adv_util_y1) + self.adv_accuracy(self.util_adv, d2, adv_util_y2)) / 2
            utility_acc_stat = (self.adv_accuracy(self.util_adv, s1, adv_util_y1) + self.adv_accuracy(self.util_adv, s2, adv_util_y2)) / 2
            if verbose: print('Utility Accuracy Dynamic: ', utility_acc_dyn.item(), '\tUtility Accuracy Static: ', utility_acc_stat.item())

        if self.use_adv and discrim_adv:
            # discrimnator (adversarial)
            discrim_out_fake = self.discriminator(torch.cat((x1_hat, x2_hat, y1_hat, y2_hat, x1_hat_, x2_hat_, y1_hat_, y2_hat_)))
            discriminator_loss = self.bce_loss(discrim_out_fake, torch.ones_like(discrim_out_fake))
            discriminator_acc = torch.sum(torch.round(discrim_out_fake) == 0).float() / (8 * batch_size)
            if verbose: print('Discriminator Loss: ', discriminator_loss.item(), '\tDiscriminator Accuracy: ', discriminator_acc.item())

        losses = {
            'rec_loss': rec_loss.item(),
            'cross_loss': cross_loss.item(),
            'end_effector_loss': end_effector_loss.item(),
            'triplet_loss': triplet_loss.item(),
            'latent_consistency_loss': latent_consistency_loss.item(),
            'privacy_loss': privacy_loss.item(),
            'privacy_loss_dyn': privacy_loss_dyn.item(),
            'privacy_loss_stat': privacy_loss_stat.item(),
            'privacy_acc_dyn': privacy_acc_dyn.item(),
            'privacy_acc_stat': privacy_acc_stat.item(),
            'utility_loss': utility_loss.item(),
            'utility_loss_dyn': utility_loss_dyn.item(),
            'utility_loss_stat': utility_loss_stat.item(),
            'utility_acc_dyn': utility_acc_dyn.item(),
            'utility_acc_stat': utility_acc_stat.item(),
            'discriminator_loss': discriminator_loss.item(),
            'discriminator_acc': discriminator_acc.item()
        }

        return rec_loss * self.lambda_rec \
                + cross_loss * self.lambda_cross \
                + end_effector_loss * self.lambda_ee \
                + triplet_loss * self.lambda_trip \
                + latent_consistency_loss * self.lambda_latent \
                + privacy_loss * self.lambda_adv_priv \
                + utility_loss * self.lambda_adv_util \
                + discriminator_loss * self.lambda_adv_disc, \
                x1_hat, x2_hat, y1_hat, y2_hat, losses

    def loss_unpaired(self, x_pos, x_rot, actors, actions, reconstruction = True, emb_adv = False, discrim_adv = False, ee = False, triplet = False, verbose = False):
        d = self.dynamic_encoder(x_rot)
        s = self.static_encoder(x_pos)
        x_hat = self.decoder(torch.cat((d, s), dim=1))

        if not one_dimension_conv:
            x = x_pos.reshape(x_pos.size(0), T, -1)

        # initialize all losses to 0 tensor
        rec_loss = torch.zeros(1).to(device)
        end_effector_loss = torch.zeros(1).to(device)
        triplet_loss = torch.zeros(1).to(device)
        privacy_loss = torch.zeros(1).to(device)
        privacy_loss_dyn = torch.zeros(1).to(device)
        privacy_loss_stat = torch.zeros(1).to(device)
        privacy_acc_dyn = torch.zeros(1).to(device)
        privacy_acc_stat = torch.zeros(1).to(device)
        utility_loss = torch.zeros(1).to(device)
        utility_loss_dyn = torch.zeros(1).to(device)
        utility_loss_stat = torch.zeros(1).to(device)
        utility_acc_dyn = torch.zeros(1).to(device)
        utility_acc_stat = torch.zeros(1).to(device)
        discriminator_loss = torch.zeros(1).to(device)
        discriminator_acc = torch.zeros(1).to(device)

        # Reconstruction Loss
        if self.use_rec_loss and reconstruction:
            rec_loss = self.reconstruction_loss(x, x_hat)
            if verbose: print('Reconstruction Loss: ', rec_loss.item())

        # End Effector Loss
        if self.use_ee_loss and ee:
            end_effector_loss = self.end_effector_loss(x_hat, x)
            if verbose: print('End Effector Loss: ', end_effector_loss.item())

        # Triplet Loss
        if self.use_trip_loss and triplet:
            triplet_loss = 1 # idk what to use for anchor
            if verbose: print('Triplet Loss: ', triplet_loss.item())

        # Adversarial Loss
        if self.use_adv and emb_adv:
            # latent privacy loss (adversarial)
            adv_priv_y = actors - 1
            adv_priv_y = torch.eye(privacy_classes)[adv_priv_y.long()].to(device)
            privacy_loss_dyn = -(self.adv_loss(self.priv_adv, d, adv_priv_y))
            privacy_loss_stat = (self.adv_loss(self.priv_adv, s, adv_priv_y))
            privacy_loss = privacy_loss_dyn + privacy_loss_stat
            if verbose: print('Privacy Loss Dynamic: ', privacy_loss_dyn.item(), '\tPrivacy Loss Static: ', privacy_loss_stat.item(), '\tPrivacy Loss: ', privacy_loss.item())

            privacy_acc_dyn = (self.adv_accuracy(self.priv_adv, d, adv_priv_y))
            privacy_acc_stat = (self.adv_accuracy(self.priv_adv, s, adv_priv_y))
            if verbose: print('Privacy Accuracy Dynamic: ', privacy_acc_dyn.item(), '\tPrivacy Accuracy Static: ', privacy_acc_stat.item())
            
            # latent utility loss (adversarial)
            adv_util_y = actions - 1
            adv_util_y = torch.eye(utility_classes)[adv_util_y.long()].to(device)
            utility_loss_dyn = (self.adv_loss(self.util_adv, d, adv_util_y))
            utility_loss_stat = -(self.adv_loss(self.util_adv, s, adv_util_y))
            utility_loss = utility_loss_dyn + utility_loss_stat
            if verbose: print('Utility Loss Dynamic: ', utility_loss_dyn.item(), '\tUtility Loss Static: ', utility_loss_stat.item(), '\tUtility Loss: ', utility_loss.item())

            utility_acc_dyn = (self.adv_accuracy(self.util_adv, d, adv_util_y))
            utility_acc_stat = (self.adv_accuracy(self.util_adv, s, adv_util_y))
            if verbose: print('Utility Accuracy Dynamic: ', utility_acc_dyn.item(), '\tUtility Accuracy Static: ', utility_acc_stat.item())

        if self.use_adv and discrim_adv:
            # discrimnator (adversarial)
            discrim_out_fake = self.discriminator(x_hat)
            discriminator_loss = self.bce_loss(discrim_out_fake, torch.ones_like(discrim_out_fake))
            discriminator_acc = torch.sum(torch.round(discrim_out_fake) == 0).float() / (batch_size)
            if verbose: print('Discriminator Loss: ', discriminator_loss.item(), '\tDiscriminator Accuracy: ', discriminator_acc.item())

        losses = {
            'rec_loss': rec_loss.item(),
            'end_effector_loss': end_effector_loss.item(),
            'triplet_loss': triplet_loss.item(),
            'privacy_loss': privacy_loss.item(),
            'privacy_loss_dyn': privacy_loss_dyn.item(),
            'privacy_loss_stat': privacy_loss_stat.item(),
            'privacy_acc_dyn': privacy_acc_dyn.item(),
            'privacy_acc_stat': privacy_acc_stat.item(),
            'utility_loss': utility_loss.item(),
            'utility_loss_dyn': utility_loss_dyn.item(),
            'utility_loss_stat': utility_loss_stat.item(),
            'utility_acc_dyn': utility_acc_dyn.item(),
            'utility_acc_stat': utility_acc_stat.item(),
            'discriminator_loss': discriminator_loss.item(),
            'discriminator_acc': discriminator_acc.item()
        }

        return rec_loss * self.lambda_rec \
                + end_effector_loss * self.lambda_ee \
                + triplet_loss * self.lambda_trip \
                + privacy_loss * self.lambda_adv_priv \
                + utility_loss * self.lambda_adv_util \
                + discriminator_loss * self.lambda_adv_disc, \
                x_hat, losses

    def reconstruction_loss(self, x, y):
        # return torch.square(torch.norm(x - y, dim=1)).mean()
        return F.mse_loss(x, y)
    
    def cross_loss(self, x1, x2, y1, y2):
        # return torch.square(torch.norm(x1 - y2, dim=1)).mean() + torch.square(torch.norm(x2 - y1, dim=1)).mean()
        return F.mse_loss(x1, y1) + F.mse_loss(x2, y2)
    
    def latent_consistency_loss(self, x, y):
        return F.mse_loss(x, y)
    
    def end_effector_loss(self, x, y):
        # slice to get the end effector joints
        x_ee = x[:, :, self.end_effectors.unsqueeze(-1) + torch.arange(3).to(device)] 
        y_ee = y[:, :, self.end_effectors.unsqueeze(-1) + torch.arange(3).to(device)]

        # calculate velocities
        x_vel = torch.norm(x_ee[:, 1:] - x_ee[:, :-1], dim=-1) / self.chain_lengths.unsqueeze(0)
        y_vel = torch.norm(y_ee[:, 1:] - y_ee[:, :-1], dim=-1) / self.chain_lengths.unsqueeze(0)
        
        # compute mse loss for each joint
        losses = F.mse_loss(x_vel, y_vel, reduction='none')

        # take sum over end effectors
        loss = losses.sum(dim=1)

        # take mean over batch
        loss = loss.mean()
        
        return loss
    
    def adv_loss(self, model, x, y):
        return F.cross_entropy(model(x), y)
    
    def adv_accuracy(self, model, x, y):
        return (model(x).argmax(dim=1) == y.argmax(dim=1)).float().mean()

    def train_adv_paired(self, x1, x1_rot, x2, x2_rot, y1, y1_rot, y2, y2_rot, actors, actions, train_emb = True, train_discrim = True):
        if not self.use_adv: return 0,0
        # freeze encoders/decoder
        self.dynamic_encoder.eval()
        self.static_encoder.eval()
        self.decoder.eval()

        # unfreeze adversaries
        self.priv_adv.train()
        self.util_adv.train()
        self.discriminator.train()

        # zero out gradients
        self.priv_optim.zero_grad()
        self.util_optim.zero_grad()
        self.discriminator_optim.zero_grad()

        # encode
        d1 = self.dynamic_encoder(x1_rot)
        d2 = self.dynamic_encoder(x2_rot)
        d3 = self.dynamic_encoder(y1_rot)
        d4 = self.dynamic_encoder(y2_rot)
        s1 = self.static_encoder(x1)
        s2 = self.static_encoder(x2)
        s3 = self.static_encoder(y1)
        s4 = self.static_encoder(y2)

        # decode
        x1_hat = self.decoder(torch.cat((d1, s1), dim=1))
        x2_hat = self.decoder(torch.cat((d2, s2), dim=1))
        y1_hat = self.decoder(torch.cat((d3, s3), dim=1))
        y2_hat = self.decoder(torch.cat((d4, s4), dim=1))

        # instantiate losses
        priv_loss = torch.zeros(1).to(device)
        util_loss = torch.zeros(1).to(device)
        discriminator_loss = torch.zeros(1).to(device)

        if train_emb:
            # train privacy adversary
            p1, p2 = actors[0] - 1, actors[1] - 1
            p1, p2 = torch.eye(privacy_classes)[p1.long()].to(device), torch.eye(privacy_classes)[p2.long()].to(device)
            priv_loss = F.cross_entropy(self.priv_adv(s1), p1) + F.cross_entropy(self.priv_adv(s2), p2) + F.cross_entropy(self.priv_adv(s3), p2) + F.cross_entropy(self.priv_adv(s4), p1)
            priv_loss.backward(retain_graph=True)
            self.priv_optim.step()
            
            # train utility adversary
            a1, a2 = actions[0] - 1, actions[1] - 1
            a1, a2 = torch.eye(utility_classes)[a1.long()].to(device), torch.eye(utility_classes)[a2.long()].to(device)
            util_loss = F.cross_entropy(self.util_adv(d1), a1) + F.cross_entropy(self.util_adv(d2), a2) + F.cross_entropy(self.util_adv(d3), a1) + F.cross_entropy(self.util_adv(d4), a2)
            util_loss.backward(retain_graph=True)
            self.util_optim.step()

        if train_discrim:
            # train discriminator
            output_real = self.discriminator(torch.cat((x1.view(x1.size(0), T, -1), x2.view(x2.size(0), T, -1), y1.view(y1.size(0), T, -1), y2.view(y1.size(0), T, -1))))
            output_fake = self.discriminator(torch.cat((x1_hat, x2_hat, y1_hat, y2_hat)))
            discriminator_loss = self.bce_loss(output_real, torch.ones_like(output_real)) + self.bce_loss(output_fake, torch.zeros_like(output_fake))
            discriminator_loss.backward()
            self.discriminator_optim.step()

        # unfreeze encoders/decoder
        self.dynamic_encoder.train()
        self.static_encoder.train()
        self.decoder.train()

        # freeze adversaries
        self.priv_adv.eval()
        self.util_adv.eval()
        self.discriminator.eval()

        return priv_loss.item(), util_loss.item(), discriminator_loss.item()

    def train_adv_unpaired(self, x_pos, x_rot, actor, action, train_emb = True, train_discrim = True):
        # ensure one training method is enabled
        assert train_emb or train_discrim, 'At least one training method must be enabled'

        # freeze encoders/decoder
        self.dynamic_encoder.eval()
        self.static_encoder.eval()
        self.decoder.eval()

        # unfreeze adversaries
        self.priv_adv.train()
        self.util_adv.train()
        self.discriminator.train()

        # zero out gradients
        self.priv_optim.zero_grad()
        self.util_optim.zero_grad()
        self.discriminator_optim.zero_grad()

        # encode
        d = self.dynamic_encoder(x_rot)
        s = self.static_encoder(x_pos)

        # decode
        x_hat = self.decoder(torch.cat((d, s), dim=1))

        # instantiate losses
        priv_loss = torch.zeros(1).to(device)
        util_loss = torch.zeros(1).to(device)
        discriminator_loss = torch.zeros(1).to(device)

        if train_emb:
            # train privacy adversary
            p = actor - 1
            p = torch.eye(privacy_classes)[p.long()].to(device)
            priv_loss = F.cross_entropy(self.priv_adv(s), p)
            priv_loss.backward(retain_graph=True)
            self.priv_optim.step()
            
            # train utility adversary
            a = action - 1
            a = torch.eye(utility_classes)[a.long()].to(device)
            util_loss = F.cross_entropy(self.util_adv(d), a)
            util_loss.backward(retain_graph=True)
            self.util_optim.step()

        if train_discrim:
            # train discriminator
            output_real = self.discriminator(x_pos.reshape(x_pos.size(0), T, -1))
            output_fake = self.discriminator(x_hat)
            discriminator_loss = self.bce_loss(output_real, torch.ones_like(output_real)) + self.bce_loss(output_fake, torch.zeros_like(output_fake))
            discriminator_loss.backward()
            self.discriminator_optim.step()

        # unfreeze encoders/decoder
        self.dynamic_encoder.train()
        self.static_encoder.train()
        self.decoder.train()

        # freeze adversaries
        self.priv_adv.eval()
        self.util_adv.eval()
        self.discriminator.eval()

        return priv_loss.item(), util_loss.item(), discriminator_loss.item()

    def forward(self, x, x_rot):
        dyn = self.dynamic_encoder(x_rot)
        sta = self.static_encoder(x)
        x = self.decoder(torch.cat((dyn, sta), dim=1))
        return x

## Utility/Privacy Evaluation

In [53]:
def test(test_loader, model, k=3):
    acces = AverageMeter()
    topk_acces = AverageMeter()
    # load learnt model that obtained best performance on validation set
    model.eval()

    label_output = list()
    pred_output = list()

    for i, t in enumerate(test_loader):
        inputs = t[0]
        target = t[1]
        with torch.no_grad():
            output = model(inputs.cuda())
            output = output.view(
                (-1, inputs.size(0)//target.size(0), output.size(1)))
            output = output.mean(1)

        label_output.append(target.cpu().numpy())
        pred_output.append(output.cpu().numpy())

        acc = accuracy(output.data, target.cuda())
        acces.update(acc[0], inputs.size(0))
        topk_acc = top_k_accuracy(output.data, target.cuda(), k=3)
        topk_acces.update(topk_acc[0], inputs.size(0))

    label_output = np.concatenate(label_output, axis=0)
    pred_output = np.concatenate(pred_output, axis=0)

    label_index = np.argmax(label_output, axis=1)
    pred_index = np.argmax(pred_output, axis=1)

    f1 = f1_score(label_index, pred_index, average='macro', zero_division=0)
    precision = precision_score(label_index, pred_index, average='macro', zero_division=0)
    recall = recall_score(label_index, pred_index, average='macro', zero_division=0)

    return acces.avg, f1, precision, recall, topk_acces.avg
    
def accuracy(output, target):
    batch_size = target.size(0)
    _, pred = output.topk(1, 1, True, True)
    pred = pred.t()
    target = torch.argmax(target, dim=1)  # Add this line to convert one-hot targets to class indices
    correct = pred.eq(target.view(1, -1).expand_as(pred))
    correct = correct.view(-1).float().sum(0, keepdim=True)
    return correct.mul_(100.0 / batch_size)

def top_k_accuracy(output, target, k=3):
    batch_size = target.size(0)
    _, pred = output.topk(k, 1, True, True) 
    pred = pred.t()
    target = torch.argmax(target, dim=1) 
    correct = pred.eq(target.view(1, -1).expand_as(pred))
    correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
    return correct_k.mul_(100.0 / batch_size)
    
def run_sgn_eval(train_x, train_y, test_x, test_y, val_x, val_y, case, model, k=3):
    # Data loading
    ntu_loaders = NTUDataLoaders(dataset, case, seg=20, train_X=train_x, train_Y=train_y, test_X=test_x, test_Y=test_y, val_X=val_x, val_Y=val_y, aug=0)
    test_loader = ntu_loaders.get_test_loader(batch_size, 16)

    # Test
    return test(test_loader, model, k=k)

## Instantiate Models

In [54]:
model = AutoEncoder().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
# scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.99)

In [55]:
load_model = True
if load_model:
    # model.load_state_dict(torch.load('pretrained/MR.pt'))
    model.load_state_dict(torch.load('model.pt'))

In [56]:
load_util = True
train_util = False
sgn_ar = SGN(utility_classes, None, seg, batch_size, 0).to(device)
sgn_priv = SGN(privacy_classes, None, seg, batch_size, 0).to(device)

if load_util:
    if ntu_120:
        sgn_priv.load_state_dict(torch.load('SGN/pretrained/privacy.pt')['state_dict'])
        sgn_ar.load_state_dict(torch.load('SGN/pretrained/action.pt')['state_dict'])
    else:
        sgn_priv.load_state_dict(torch.load('SGN/pretrained/privacy_60.pt')['state_dict'])
        sgn_ar.load_state_dict(torch.load('SGN/pretrained/action_60.pt')['state_dict'])

# Training

## Train Motion Retargeting

In [57]:
sgn_train_x, sgn_train_y, sgn_val_x, sgn_val_y = np.zeros((batch_size, 300, 150)), np.zeros((batch_size, 1)), np.zeros((batch_size, 300, 150)), np.zeros((batch_size, 1))

best_loss = float('inf')
total_epochs = -1
cur_tot_epoch = 0

def train_paired(epoch, train_ae = True, train_cross = True, train_discrim = True, train_emb_adv = True, run_eval = True, use_emb_adv = True, use_discrim_adv = True, run_sgn_eval = False, save = True, k=3):
    global best_loss
    global total_epochs
    global cur_tot_epoch
    # Assertions
    assert train_ae or train_cross or train_discrim or train_emb_adv, "At least one of the training objectives must be True"
    assert not (run_sgn_eval and not run_eval), "If run_sgn_eval is True, then run_eval must be True"
    
    # Store eval values for validation
    eval_X_known, eval_Y_known_action, eval_Y_known_actor, eval_X_rec, eval_Y_rec_action, eval_Y_rec_actor, eval_X, eval_Y_action, eval_Y_actor = [], [], [], [], [], [], [], [], []

    # Losses for printing
    losses = []
    rec_loss, cross_loss, end_effector_loss, triplet_loss, latent_consistency_loss, privacy_loss, privacy_loss_dyn, privacy_loss_stat, privacy_acc_dyn, privacy_acc_stat, priv_training_loss, utility_loss, utility_loss_dyn, utility_loss_stat, utility_acc_dyn, utility_acc_stat, util_training_loss, discriminator_loss, discriminator_train_losses, discriminator_training_acc = [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []

    for (x1_pos, x1_rot, x2_pos, x2_rot, y1_pos, y1_rot, y2_pos, y2_rot, actors, actions) in train_dl:
        # Move tensors to the configured device
        x1_pos, x1_rot, x2_pos, x2_rot, y1_pos, y1_rot, y2_pos, y2_rot = x1_pos.float().to(device), x1_rot.float().to(device), x2_pos.float().to(device), x2_rot.float().to(device), y1_pos.float().to(device), y1_rot.float().to(device), y2_pos.float().to(device), y2_rot.float().to(device)
        
        # For 1D convolutions, flatten the data
        if one_dimension_conv:
            x1_pos = x1_pos.view(x1_pos.size(0), T, -1)
            x1_rot = x1_rot.view(x1_rot.size(0), T, -1)
            x2_pos = x2_pos.view(x2_pos.size(0), T, -1)
            x2_rot = x2_rot.view(x2_rot.size(0), T, -1)
            y1_pos = y1_pos.view(y1_pos.size(0), T, -1)
            y1_rot = y1_rot.view(y1_rot.size(0), T, -1)
            y2_pos = y2_pos.view(y2_pos.size(0), T, -1)
            y2_rot = y2_rot.view(y2_rot.size(0), T, -1)

        
        if train_discrim or train_emb_adv:
            # Train the discriminator
            priv_train_loss, util_train_loss, discriminator_train_loss = model.train_adv_paired(x1_pos, x1_rot, x2_pos, x2_rot, y1_pos, y1_rot, y2_pos, y2_rot, actors, actions, train_emb=train_emb_adv, train_discrim=train_discrim)
            
            # Track the loss
            priv_training_loss.append(priv_train_loss)
            util_training_loss.append(util_train_loss)
            discriminator_train_losses.append(discriminator_train_loss)

        # Zero the gradients
        optimizer.zero_grad()

        # Train the autoencoder/cross reconstruction
        if train_ae or train_cross:
            # Forward pass
            loss, _, _, _, _, losses_ = model.loss_paired(x1_pos, x1_rot, x2_pos, x2_rot, y1_pos, y1_rot, y2_pos, y2_rot, actors, actions, cross=train_cross, reconstruction=train_ae, emb_adv=use_emb_adv, discrim_adv=use_discrim_adv)

            # Backward and optimize
            loss.backward()
            optimizer.step()

            # Track the loss
            losses.append(loss.item())
            rec_loss.append(losses_['rec_loss'])
            cross_loss.append(losses_['cross_loss'])
            end_effector_loss.append(losses_['end_effector_loss'])
            latent_consistency_loss.append(losses_['latent_consistency_loss'])
            # triplet_loss.append(losses_['triplet_loss'])
            privacy_loss.append(losses_['privacy_loss'])
            privacy_loss_dyn.append(losses_['privacy_loss_dyn'])
            privacy_loss_stat.append(losses_['privacy_loss_stat'])
            privacy_acc_dyn.append(losses_['privacy_acc_dyn'])
            privacy_acc_stat.append(losses_['privacy_acc_stat'])
            utility_loss.append(losses_['utility_loss'])
            utility_loss_dyn.append(losses_['utility_loss_dyn'])
            utility_loss_stat.append(losses_['utility_loss_stat'])
            utility_acc_dyn.append(losses_['utility_acc_dyn'])
            utility_acc_stat.append(losses_['utility_acc_stat'])
            discriminator_loss.append(losses_['discriminator_loss'])
            discriminator_training_acc.append(losses_['discriminator_acc'])
        
    # Decay learning rate (disabled for training stages)
    # scheduler.step() 

    # Validation
    if run_eval:
        with torch.no_grad():
            val_losses = []
            val_rec_loss, val_cross_loss, val_end_effector_loss, val_triplet_loss, val_latent_consistency_loss, val_privacy_loss, val_privacy_loss_dyn, val_privacy_loss_stat, val_privacy_acc_dyn, val_privacy_acc_stat, val_utility_loss, val_utility_loss_dyn, val_utility_loss_stat, val_utility_acc_dyn, val_utility_acc_stat, val_discriminator_loss, val_discriminator_acc = [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []
            
            for (x1_pos, x1_rot, x2_pos, x2_rot, y1_pos, y1_rot, y2_pos, y2_rot, actors, actions) in val_dl:
                x1_pos, x1_rot, x2_pos, x2_rot, y1_pos, y1_rot, y2_pos, y2_rot = x1_pos.float().to(device), x1_rot.float().to(device), x2_pos.float().to(device), x2_rot.float().to(device), y1_pos.float().to(device), y1_rot.float().to(device), y2_pos.float().to(device), y2_rot.float().to(device)

                # For 1D convolutions, flatten the data
                if one_dimension_conv:
                    x1_pos = x1_pos.view(x1_pos.size(0), T, -1)
                    x1_rot = x1_rot.view(x1_rot.size(0), T, -1)
                    x2_pos = x2_pos.view(x2_pos.size(0), T, -1)
                    x2_rot = x2_rot.view(x2_rot.size(0), T, -1)
                    y1_pos = y1_pos.view(y1_pos.size(0), T, -1)
                    y1_rot = y1_rot.view(y1_rot.size(0), T, -1)
                    y2_pos = y2_pos.view(y2_pos.size(0), T, -1)
                    y2_rot = y2_rot.view(y2_rot.size(0), T, -1)
                
                loss, x1_hat, x2_hat, y1_hat, y2_hat, losses_ = model.loss_paired(x1_pos, x1_rot, x2_pos, x2_rot, y1_pos, y1_rot, y2_pos, y2_rot, actors, actions, cross=train_cross, reconstruction=train_ae, emb_adv=use_emb_adv, discrim_adv=use_discrim_adv)
                val_losses.append(loss.item())
                val_rec_loss.append(losses_['rec_loss'])
                val_cross_loss.append(losses_['cross_loss'])
                val_end_effector_loss.append(losses_['end_effector_loss'])
                # val_triplet_loss.append(losses_['triplet_loss'])
                val_latent_consistency_loss.append(losses_['latent_consistency_loss'])
                val_privacy_loss.append(losses_['privacy_loss'])
                val_privacy_loss_dyn.append(losses_['privacy_loss_dyn'])
                val_privacy_loss_stat.append(losses_['privacy_loss_stat'])
                val_privacy_acc_dyn.append(losses_['privacy_acc_dyn'])
                val_privacy_acc_stat.append(losses_['privacy_acc_stat'])
                val_utility_loss.append(losses_['utility_loss'])
                val_utility_loss_dyn.append(losses_['utility_loss_dyn'])
                val_utility_loss_stat.append(losses_['utility_loss_stat'])
                val_utility_acc_dyn.append(losses_['utility_acc_dyn'])
                val_utility_acc_stat.append(losses_['utility_acc_stat'])
                val_discriminator_loss.append(losses_['discriminator_loss'])
                val_discriminator_acc.append(losses_['discriminator_acc'])

                if run_sgn_eval:
                    if not one_dimension_conv:
                        x1_pos = x1_pos.view(x1_pos.size(0), T, -1)
                        x2_pos = x2_pos.view(x2_pos.size(0), T, -1)
                        y1_pos = y1_pos.view(y1_pos.size(0), T, -1)
                        y2_pos = y2_pos.view(y2_pos.size(0), T, -1)
                        
                    eval_X_known.append(x1_pos.cpu().numpy())
                    eval_X_known.append(x2_pos.cpu().numpy())
                    eval_X_known.append(y1_pos.cpu().numpy())
                    eval_X_known.append(y2_pos.cpu().numpy())

                    eval_Y_known_action.append(actions[0].cpu().numpy())
                    eval_Y_known_action.append(actions[1].cpu().numpy())
                    eval_Y_known_action.append(actions[0].cpu().numpy())
                    eval_Y_known_action.append(actions[1].cpu().numpy())

                    eval_Y_known_actor.append(actors[0].cpu().numpy())
                    eval_Y_known_actor.append(actors[1].cpu().numpy())
                    eval_Y_known_actor.append(actors[1].cpu().numpy())
                    eval_Y_known_actor.append(actors[0].cpu().numpy())

                    eval_X_rec.append(x1_hat.cpu().numpy())
                    eval_X_rec.append(x2_hat.cpu().numpy())
                    eval_X.append(y1_hat.cpu().numpy())
                    eval_X.append(y2_hat.cpu().numpy())

                    eval_Y_rec_action.append(actions[0].cpu().numpy())
                    eval_Y_rec_action.append(actions[1].cpu().numpy())
                    eval_Y_action.append(actions[0].cpu().numpy())
                    eval_Y_action.append(actions[1].cpu().numpy())

                    eval_Y_rec_actor.append(actors[0].cpu().numpy())
                    eval_Y_rec_actor.append(actors[1].cpu().numpy())
                    eval_Y_actor.append(actors[1].cpu().numpy())
                    eval_Y_actor.append(actors[0].cpu().numpy())

    # Print loss/accuracy
    print(f'--------------------\nEpoch {cur_tot_epoch+1}/{total_epochs}\n--------------------')
    cur_tot_epoch += 1
    
    if train_ae or train_cross:
        print(f'Training Loss:\t\t\t{np.mean(losses)}\nValidation Loss:\t\t{np.mean(val_losses)}')
        print('\nTraining Losses:')
        print(f'Reconstruction Loss:\t\t{np.mean(rec_loss)}\nCross Reconstruction Loss:\t{np.mean(cross_loss)}\nEnd Effector Loss:\t\t{np.mean(end_effector_loss)}\nTriplet Loss:\t\t\t{np.mean(triplet_loss)}\nLatent Consistency Loss:\t{np.mean(latent_consistency_loss)}')
        print(f'Privacy Loss:\t\t\t{np.mean(privacy_loss)}\nPrivacy Loss Dyn:\t\t{np.mean(privacy_loss_dyn)}\nPrivacy Loss Stat:\t\t{np.mean(privacy_loss_stat)}')
        print(f'Utility Loss:\t\t\t{np.mean(utility_loss)}\nUtility Loss Dyn:\t\t{np.mean(utility_loss_dyn)}\nUtility Loss Stat:\t\t{np.mean(utility_loss_stat)}')
        print(f'Discriminator Loss:\t\t{np.mean(discriminator_loss)}')

    if run_eval:
        print('\nValidation Losses:')
        print(f'Val Reconstruction Loss:\t{np.mean(val_rec_loss)}\nVal Cross Reconstruction Loss:\t{np.mean(val_cross_loss)}\nVal End Effector Loss:\t\t{np.mean(val_end_effector_loss)}\nVal Triplet Loss:\t\t{np.mean(val_triplet_loss)}\nVal Latent Consistency Loss:\t{np.mean(val_latent_consistency_loss)}')
        print(f'Val Privacy Loss:\t\t{np.mean(val_privacy_loss)}\nVal Privacy Loss Dyn:\t\t{np.mean(val_privacy_loss_dyn)}\nVal Privacy Loss Stat:\t\t{np.mean(val_privacy_loss_stat)}')
        print(f'Val Utility Loss:\t\t{np.mean(val_utility_loss)}\nVal Utility Loss Dyn:\t\t{np.mean(val_utility_loss_dyn)}\nVal Utility Loss Stat:\t\t{np.mean(val_utility_loss_stat)}')
        print(f'Val Discriminator Loss:\t\t{np.mean(val_discriminator_loss)}')
    
    if train_emb_adv or train_discrim:
        print('\nAdversary Losses')
        if train_emb_adv:
            print(f'Privacy Training Loss:\t\t{np.mean(priv_training_loss)}\nUtility Training Loss:\t\t{np.mean(util_training_loss)}\nDiscriminator Training Loss:\t{np.mean(discriminator_train_losses)}')
            if train_ae or train_cross:
                print(f'Privacy Acc Dyn:\t\t{np.mean(privacy_acc_dyn)}\nPrivacy Acc Stat:\t\t{np.mean(privacy_acc_stat)}')
                print(f'Utility Acc Dyn:\t\t{np.mean(utility_acc_dyn)}\nUtility Acc Stat:\t\t{np.mean(utility_acc_stat)}')
            print(f'Val Privacy Acc Dyn:\t\t{np.mean(val_privacy_acc_dyn)}\nVal Privacy Acc Stat:\t\t{np.mean(val_privacy_acc_stat)}')
            print(f'Val Utility Acc Dyn:\t\t{np.mean(val_utility_acc_dyn)}\nVal Utility Acc Stat:\t\t{np.mean(val_utility_acc_stat)}')
    
    if train_ae or train_cross: print(f'Discriminator Acc:\t\t{np.mean(discriminator_training_acc)}')
    if run_eval: print(f'Val Discriminator Acc:\t\t{np.mean(val_discriminator_acc)}')

    # Save model
    if save and np.mean(val_losses) < best_loss:
        best_loss = np.mean(val_losses)
        torch.save(model.state_dict(), 'pretrained/MR.pt')

    # Test Accuracy
    if run_sgn_eval and run_eval:
        print('\n')
        sgn_acc_known_acc, sgn_acc_known_f1, sgn_acc_known_prec, sgn_acc_known_recall, sgn_acc_known_topk = sgn_eval(eval_X_known, eval_Y_known_action, 'Known Action', is_action=True, k=k)
        sgn_acc_rec_acc, sgn_acc_rec_f1, sgn_acc_rec_prec, sgn_acc_rec_recall, sgn_acc_rec_topk = sgn_eval(eval_X_rec, eval_Y_rec_action, 'Reconstructed Action', is_action=True, k=k)
        sgn_acc_cross_acc, sgn_acc_cross_f1, sgn_acc_cross_prec, sgn_acc_cross_recall, sgn_acc_cross_topk = sgn_eval(eval_X, eval_Y_action, 'Generated Action', is_action=True, k=k)
        print('\n')
        sgn_priv_known_acc, sgn_priv_known_f1, sgn_priv_known_prec, sgn_priv_known_recall, sgn_priv_known_topk = sgn_eval(eval_X_known, eval_Y_known_actor, 'Known Actor', is_actor=True, k=k)
        sgn_priv_rec_acc, sgn_priv_rec_f1, sgn_priv_rec_prec, sgn_priv_rec_recall, sgn_priv_rec_topk = sgn_eval(eval_X_rec, eval_Y_rec_actor, 'Reconstructed Actor', is_actor=True, k=k)
        sgn_priv_cross_acc, sgn_priv_cross_f1, sgn_priv_cross_prec, sgn_priv_cross_recall, sgn_priv_cross_topk = sgn_eval(eval_X, eval_Y_actor, 'Generated Actor', is_actor=True, k=k)
    else: print('\n')

    # Return dict with all losses and accuracies for plotting
    losses_dict = {}
    if train_ae or train_cross:
        losses_dict['loss'] = np.mean(losses)
        losses_dict['val_loss'] = np.mean(val_losses)
        losses_dict['rec_loss'] = np.mean(rec_loss)
        losses_dict['cross_loss'] = np.mean(cross_loss)
        losses_dict['end_effector_loss'] = np.mean(end_effector_loss)
        losses_dict['triplet_loss'] = np.mean(triplet_loss)
        losses_dict['latent_consistency_loss'] = np.mean(latent_consistency_loss)
        losses_dict['privacy_loss'] = np.mean(privacy_loss)
        losses_dict['privacy_loss_dyn'] = np.mean(privacy_loss_dyn)
        losses_dict['privacy_loss_stat'] = np.mean(privacy_loss_stat)
        losses_dict['utility_loss'] = np.mean(utility_loss)
        losses_dict['utility_loss_dyn'] = np.mean(utility_loss_dyn)
        losses_dict['utility_loss_stat'] = np.mean(utility_loss_stat)
        losses_dict['discriminator_loss'] = np.mean(discriminator_loss)
    if run_eval:
        losses_dict['val_rec_loss'] = np.mean(val_rec_loss)
        losses_dict['val_cross_loss'] = np.mean(val_cross_loss)
        losses_dict['val_end_effector_loss'] = np.mean(val_end_effector_loss)
        losses_dict['val_triplet_loss'] = np.mean(val_triplet_loss)
        losses_dict['val_latent_consistency_loss'] = np.mean(val_latent_consistency_loss)
        losses_dict['val_privacy_loss'] = np.mean(val_privacy_loss)
        losses_dict['val_privacy_loss_dyn'] = np.mean(val_privacy_loss_dyn)
        losses_dict['val_privacy_loss_stat'] = np.mean(val_privacy_loss_stat)
        losses_dict['val_utility_loss'] = np.mean(val_utility_loss)
        losses_dict['val_utility_loss_dyn'] = np.mean(val_utility_loss_dyn)
        losses_dict['val_utility_loss_stat'] = np.mean(val_utility_loss_stat)
        losses_dict['val_discriminator_loss'] = np.mean(val_discriminator_loss)
    if train_emb_adv or train_discrim:
        losses_dict['priv_training_loss'] = np.mean(priv_training_loss)
        losses_dict['util_training_loss'] = np.mean(util_training_loss)
        losses_dict['discriminator_train_loss'] = np.mean(discriminator_train_losses)
        if train_ae or train_cross:
            losses_dict['privacy_acc_dyn'] = np.mean(privacy_acc_dyn)
            losses_dict['privacy_acc_stat'] = np.mean(privacy_acc_stat)
            losses_dict['utility_acc_dyn'] = np.mean(utility_acc_dyn)
            losses_dict['utility_acc_stat'] = np.mean(utility_acc_stat)
        losses_dict['val_privacy_acc_dyn'] = np.mean(val_privacy_acc_dyn)
        losses_dict['val_privacy_acc_stat'] = np.mean(val_privacy_acc_stat)
        losses_dict['val_utility_acc_dyn'] = np.mean(val_utility_acc_dyn)
        losses_dict['val_utility_acc_stat'] = np.mean(val_utility_acc_stat)
    if train_ae or train_cross:
        losses_dict['discriminator_acc'] = np.mean(discriminator_training_acc)
    if run_eval:
        losses_dict['val_discriminator_acc'] = np.mean(val_discriminator_acc)
    if run_sgn_eval and run_eval:
        losses_dict['sgn_acc_known_acc'] = sgn_acc_known_acc
        losses_dict['sgn_acc_known_f1'] = sgn_acc_known_f1
        losses_dict['sgn_acc_known_prec'] = sgn_acc_known_prec
        losses_dict['sgn_acc_known_recall'] = sgn_acc_known_recall
        losses_dict['sgn_acc_rec_acc'] = sgn_acc_rec_acc
        losses_dict['sgn_acc_rec_f1'] = sgn_acc_rec_f1
        losses_dict['sgn_acc_rec_prec'] = sgn_acc_rec_prec
        losses_dict['sgn_acc_rec_recall'] = sgn_acc_rec_recall
        losses_dict['sgn_acc_cross_acc'] = sgn_acc_cross_acc
        losses_dict['sgn_acc_cross_f1'] = sgn_acc_cross_f1
        losses_dict['sgn_acc_cross_prec'] = sgn_acc_cross_prec
        losses_dict['sgn_acc_cross_recall'] = sgn_acc_cross_recall
        losses_dict['sgn_priv_known_acc'] = sgn_priv_known_acc
        losses_dict['sgn_priv_known_f1'] = sgn_priv_known_f1
        losses_dict['sgn_priv_known_prec'] = sgn_priv_known_prec
        losses_dict['sgn_priv_known_recall'] = sgn_priv_known_recall
        losses_dict['sgn_priv_rec_acc'] = sgn_priv_rec_acc
        losses_dict['sgn_priv_rec_f1'] = sgn_priv_rec_f1
        losses_dict['sgn_priv_rec_prec'] = sgn_priv_rec_prec
        losses_dict['sgn_priv_rec_recall'] = sgn_priv_rec_recall
        losses_dict['sgn_priv_cross_acc'] = sgn_priv_cross_acc
        losses_dict['sgn_priv_cross_f1'] = sgn_priv_cross_f1
        losses_dict['sgn_priv_cross_prec'] = sgn_priv_cross_prec
        losses_dict['sgn_priv_cross_recall'] = sgn_priv_cross_recall
        losses_dict['sgn_acc_known_topk'] = sgn_acc_known_topk
        losses_dict['sgn_acc_rec_topk'] = sgn_acc_rec_topk
        losses_dict['sgn_acc_cross_topk'] = sgn_acc_cross_topk
        losses_dict['sgn_priv_known_topk'] = sgn_priv_known_topk
        losses_dict['sgn_priv_rec_topk'] = sgn_priv_rec_topk
        losses_dict['sgn_priv_cross_topk'] = sgn_priv_cross_topk
    
    return losses_dict


def sgn_eval(X, Y, label='Undefined', is_actor=False, is_action=False, k=3):
    assert is_actor != is_action, "is_actor and is_action cannot both be True"
    assert is_actor or is_action, "Either is_actor or is_action must be True"

    if is_actor:
        classes = privacy_classes
        sgn = sgn_priv
    elif is_action:
        classes = utility_classes
        sgn = sgn_ar

    X = np.concatenate(X)
    X = np.pad(X, ((0,0), (0,225), (0,75)), 'constant')

    Y = np.concatenate(Y) - 1
    Y = np.eye(classes)[Y.astype(int)]

    acc, f1, prec, recall, topk = run_sgn_eval(sgn_train_x, sgn_train_y, X, Y, sgn_val_x, sgn_val_y, 1, sgn, k=k)
    print(f'\n{label} Accuracy:\t\t{acc}\n{label} F1:\t\t\t{f1*100}\n{label} Precision:\t\t{prec*100}\n{label} Recall:\t\t{recall*100}\n{label} Top-{k} Accuracy:\t{topk}\n')
    return acc, f1, prec, recall, topk

# Simplified training loop for only AE
def train_unpaired(epoch, run_eval=True, run_sgn_eval=True, save=True, ae=True, ee=False, triplet=False, use_emb_adv=False, use_discrim_adv=False, emb_adv=False, discrim_adv=False, k=3):
    global best_loss
    global total_epochs

    # Store eval values for validation
    eval_X_known, eval_Y_known_action, eval_Y_known_actor, eval_X_rec, eval_Y_rec_action, eval_Y_rec_actor = [], [], [], [], [], []
    
    # Losses for printing
    rec_loss, end_effector_loss, triplet_loss, privacy_loss, privacy_loss_dyn, privacy_loss_stat, privacy_acc_dyn, privacy_acc_stat, priv_training_loss, utility_loss, utility_loss_dyn, utility_loss_stat, utility_acc_dyn, utility_acc_stat, util_training_loss, discriminator_loss, discriminator_train_losses, discriminator_acc = [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []
    val_rec_loss, val_end_effector_loss, val_triplet_loss, val_privacy_loss, val_privacy_loss_dyn, val_privacy_loss_stat, val_privacy_acc_dyn, val_privacy_acc_stat, val_utility_loss, val_utility_loss_dyn, val_utility_loss_stat, val_utility_acc_dyn, val_utility_acc_stat, val_discriminator_loss, val_discriminator_acc = [], [], [], [], [], [], [], [], [], [], [], [], [], [], []
    losses, val_losses = [], []

    for (x, actors, actions) in rec_train_dl:
        # Move tensors to the configured device
        x = x.float().to(device)

        # Split into position and rotation
        x_pos = x[:, :, :, :3]
        x_rot = x[:, :, :, 3:]

        # Train adversaries
        if emb_adv or discrim_adv:
            # Train the discriminator
            priv_train_loss, util_train_loss, discriminator_train_loss = model.train_adv_unpaired(x_pos, x_rot, actors, actions, train_emb=emb_adv, train_discrim=discrim_adv)
            
            # Track the loss
            priv_training_loss.append(priv_train_loss)
            util_training_loss.append(util_train_loss)
            discriminator_train_losses.append(discriminator_train_loss)
        
        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        loss, _, losses_ = model.loss_unpaired(x_pos, x_rot, actors, actions, reconstruction=ae, emb_adv=use_emb_adv, discrim_adv=use_discrim_adv, ee=ee, triplet=triplet)

        # Backward and optimize
        loss.backward()
        optimizer.step()

        # Track the loss
        losses.append(loss.item())

        rec_loss.append(losses_['rec_loss'])
        end_effector_loss.append(losses_['end_effector_loss'])
        triplet_loss.append(losses_['triplet_loss'])
        privacy_loss.append(losses_['privacy_loss'])
        privacy_loss_dyn.append(losses_['privacy_loss_dyn'])
        privacy_loss_stat.append(losses_['privacy_loss_stat'])
        privacy_acc_dyn.append(losses_['privacy_acc_dyn'])
        privacy_acc_stat.append(losses_['privacy_acc_stat'])
        utility_loss.append(losses_['utility_loss'])
        utility_loss_dyn.append(losses_['utility_loss_dyn'])
        utility_loss_stat.append(losses_['utility_loss_stat'])
        utility_acc_dyn.append(losses_['utility_acc_dyn'])
        utility_acc_stat.append(losses_['utility_acc_stat'])
        discriminator_loss.append(losses_['discriminator_loss'])
        discriminator_acc.append(losses_['discriminator_acc'])


    # Decay learning rate
    # scheduler.step()

    # Validation
    if run_eval:
        with torch.no_grad():
            for (x, actors, actions) in rec_val_dl:
                x = x.float().to(device)

                # Split into position and rotation
                x_pos = x[:, :, :, :3]
                x_rot = x[:, :, :, 3:]
                
                loss, _, losses_ = model.loss_unpaired(x_pos, x_rot, actors, actions, reconstruction=ae, emb_adv=use_emb_adv, discrim_adv=use_discrim_adv, ee=ee, triplet=triplet)
                val_losses.append(loss.item())

                if run_sgn_eval:
                    eval_X_known.append(x_pos.contiguous().view(x_pos.size(0), T, -1).cpu().numpy())
                    eval_Y_known_action.append(np.array(actions))
                    eval_Y_known_actor.append(np.array(actors))

                    eval_X_rec.append(model(x_pos, x_rot).cpu().numpy())
                    eval_Y_rec_action.append(np.array(actions))
                    eval_Y_rec_actor.append(np.array(actors))
                
                val_rec_loss.append(losses_['rec_loss'])
                val_end_effector_loss.append(losses_['end_effector_loss'])
                val_triplet_loss.append(losses_['triplet_loss'])
                val_privacy_loss.append(losses_['privacy_loss'])
                val_privacy_loss_dyn.append(losses_['privacy_loss_dyn'])
                val_privacy_loss_stat.append(losses_['privacy_loss_stat'])
                val_privacy_acc_dyn.append(losses_['privacy_acc_dyn'])
                val_privacy_acc_stat.append(losses_['privacy_acc_stat'])
                val_utility_loss.append(losses_['utility_loss'])
                val_utility_loss_dyn.append(losses_['utility_loss_dyn'])
                val_utility_loss_stat.append(losses_['utility_loss_stat'])
                val_utility_acc_dyn.append(losses_['utility_acc_dyn'])
                val_utility_acc_stat.append(losses_['utility_acc_stat'])
                val_discriminator_loss.append(losses_['discriminator_loss'])
                val_discriminator_acc.append(losses_['discriminator_acc'])

    # Print loss/accuracy
    print(f'--------------------\nEpoch {epoch+1}/{total_epochs}\n--------------------')
    print(f'Training Loss:\t\t\t{np.mean(losses)}\nValidation Loss:\t\t{np.mean(val_losses)}\n')
    print('Training Losses:')
    print(f'Reconstruction Loss:\t\t{np.mean(rec_loss)}\nEnd Effector Loss:\t\t{np.mean(end_effector_loss)}\nTriplet Loss:\t\t\t{np.mean(triplet_loss)}')
    print(f'Privacy Loss:\t\t\t{np.mean(privacy_loss)}\nPrivacy Loss Dyn:\t\t{np.mean(privacy_loss_dyn)}\nPrivacy Loss Stat:\t\t{np.mean(privacy_loss_stat)}')
    print(f'Utility Loss:\t\t\t{np.mean(utility_loss)}\nUtility Loss Dyn:\t\t{np.mean(utility_loss_dyn)}\nUtility Loss Stat:\t\t{np.mean(utility_loss_stat)}')
    print(f'Discriminator Loss:\t\t{np.mean(discriminator_loss)}')
    if run_eval:
        print('\nValidation Losses:')
        print(f'Val Reconstruction Loss:\t{np.mean(val_rec_loss)}\nVal End Effector Loss:\t\t{np.mean(val_end_effector_loss)}\nVal Triplet Loss:\t\t{np.mean(val_triplet_loss)}')
        print(f'Val Privacy Loss:\t\t{np.mean(val_privacy_loss)}\nVal Privacy Loss Dyn:\t\t{np.mean(val_privacy_loss_dyn)}\nVal Privacy Loss Stat:\t\t{np.mean(val_privacy_loss_stat)}')
        print(f'Val Utility Loss:\t\t{np.mean(val_utility_loss)}\nVal Utility Loss Dyn:\t\t{np.mean(val_utility_loss_dyn)}\nVal Utility Loss Stat:\t\t{np.mean(val_utility_loss_stat)}')
        print(f'Val Discriminator Loss:\t\t{np.mean(val_discriminator_loss)}')
    if emb_adv or discrim_adv:
        print('\nAdversary Losses')
        print(f'Privacy Training Loss:\t\t{np.mean(priv_training_loss)}\nUtility Training Loss:\t\t{np.mean(util_training_loss)}\nDiscriminator Training Loss:\t{np.mean(discriminator_train_losses)}')
        if emb_adv:
            print(f'Privacy Acc Dyn:\t\t{np.mean(privacy_acc_dyn)}\nPrivacy Acc Stat:\t\t{np.mean(privacy_acc_stat)}')
            print(f'Utility Acc Dyn:\t\t{np.mean(utility_acc_dyn)}\nUtility Acc Stat:\t\t{np.mean(utility_acc_stat)}')
            print(f'Val Privacy Acc Dyn:\t\t{np.mean(val_privacy_acc_dyn)}\nVal Privacy Acc Stat:\t\t{np.mean(val_privacy_acc_stat)}')
            print(f'Val Utility Acc Dyn:\t\t{np.mean(val_utility_acc_dyn)}\nVal Utility Acc Stat:\t\t{np.mean(val_utility_acc_stat)}')
        if discrim_adv:
            print(f'Discriminator Acc:\t\t{np.mean(discriminator_acc)}')
            print(f'Val Discriminator Acc:\t\t{np.mean(val_discriminator_acc)}')

    # Save model
    if save and np.mean(val_losses) < best_loss:
        best_loss = np.mean(val_losses)
        torch.save(model.state_dict(), 'pretrained/MR.pt')

    # Test Accuracy
    if run_sgn_eval and run_eval:
        print('\n')
        sgn_acc_known_acc, sgn_acc_known_f1, sgn_acc_known_prec, sgn_acc_known_recall, sgn_acc_known_topk = sgn_eval(eval_X_known, eval_Y_known_action, 'Known Action', is_action=True, k=k)
        sgn_acc_rec_acc, sgn_acc_rec_f1, sgn_acc_rec_prec, sgn_acc_rec_recall, sgn_acc_rec_topk = sgn_eval(eval_X_rec, eval_Y_rec_action, 'Reconstructed Action', is_action=True, k=k)
        print('\n')
        sgn_priv_known_acc, sgn_priv_known_f1, sgn_priv_known_prec, sgn_priv_known_recall, sgn_priv_known_topk = sgn_eval(eval_X_known, eval_Y_known_actor, 'Known Actor', is_actor=True, k=k)
        sgn_priv_rec_acc, sgn_priv_rec_f1, sgn_priv_rec_prec, sgn_priv_rec_recall, sgn_priv_rec_topk = sgn_eval(eval_X_rec, eval_Y_rec_actor, 'Reconstructed Actor', is_actor=True, k=k)
        print('\n')
    else: print('\n')

    losses_dict = {}
    losses_dict['loss'] = np.mean(losses)

    if ae: losses_dict['rec_loss'] = np.mean(rec_loss)
    if ee: losses_dict['end_effector_loss'] = np.mean(end_effector_loss)
    if triplet: losses_dict['triplet_loss'] = np.mean(triplet_loss)
    if run_eval:
        losses_dict['val_loss'] = np.mean(val_losses)
        if ae: losses_dict['val_rec_loss'] = np.mean(val_rec_loss)
        if ee: losses_dict['val_end_effector_loss'] = np.mean(val_end_effector_loss)
        if triplet: losses_dict['val_triplet_loss'] = np.mean(val_triplet_loss)
        if run_sgn_eval:
            losses_dict['sgn_acc_known_acc'] = sgn_acc_known_acc
            losses_dict['sgn_acc_known_f1'] = sgn_acc_known_f1
            losses_dict['sgn_acc_known_prec'] = sgn_acc_known_prec
            losses_dict['sgn_acc_known_recall'] = sgn_acc_known_recall
            losses_dict['sgn_acc_rec_acc'] = sgn_acc_rec_acc
            losses_dict['sgn_acc_rec_f1'] = sgn_acc_rec_f1
            losses_dict['sgn_acc_rec_prec'] = sgn_acc_rec_prec
            losses_dict['sgn_acc_rec_recall'] = sgn_acc_rec_recall
            losses_dict['sgn_acc_known_topk'] = sgn_acc_known_topk
            losses_dict['sgn_acc_rec_topk'] = sgn_acc_rec_topk
            losses_dict['sgn_priv_known_acc'] = sgn_priv_known_acc
            losses_dict['sgn_priv_known_f1'] = sgn_priv_known_f1
            losses_dict['sgn_priv_known_prec'] = sgn_priv_known_prec
            losses_dict['sgn_priv_known_recall'] = sgn_priv_known_recall
            losses_dict['sgn_priv_rec_acc'] = sgn_priv_rec_acc
            losses_dict['sgn_priv_rec_f1'] = sgn_priv_rec_f1
            losses_dict['sgn_priv_rec_prec'] = sgn_priv_rec_prec
            losses_dict['sgn_priv_rec_recall'] = sgn_priv_rec_recall
            losses_dict['sgn_priv_known_topk'] = sgn_priv_known_topk
            losses_dict['sgn_priv_rec_topk'] = sgn_priv_rec_topk
    if use_emb_adv:
        losses_dict['privacy_loss'] = np.mean(privacy_loss)
        losses_dict['privacy_loss_dyn'] = np.mean(privacy_loss_dyn)
        losses_dict['privacy_loss_stat'] = np.mean(privacy_loss_stat)
        losses_dict['utility_loss'] = np.mean(utility_loss)
        losses_dict['utility_loss_dyn'] = np.mean(utility_loss_dyn)
        losses_dict['utility_loss_stat'] = np.mean(utility_loss_stat)
        losses_dict['privacy_acc_dyn'] = np.mean(privacy_acc_dyn)
        losses_dict['privacy_acc_stat'] = np.mean(privacy_acc_stat)
        losses_dict['utility_acc_dyn'] = np.mean(utility_acc_dyn)
        losses_dict['utility_acc_stat'] = np.mean(utility_acc_stat)
        if run_eval:    
            losses_dict['val_privacy_acc_dyn'] = np.mean(val_privacy_acc_dyn)
            losses_dict['val_privacy_acc_stat'] = np.mean(val_privacy_acc_stat)
            losses_dict['val_utility_acc_dyn'] = np.mean(val_utility_acc_dyn)
            losses_dict['val_utility_acc_stat'] = np.mean(val_utility_acc_stat)
            losses_dict['val_privacy_loss'] = np.mean(val_privacy_loss)
            losses_dict['val_privacy_loss_dyn'] = np.mean(val_privacy_loss_dyn)
            losses_dict['val_privacy_loss_stat'] = np.mean(val_privacy_loss_stat)
            losses_dict['val_utility_loss'] = np.mean(val_utility_loss)
            losses_dict['val_utility_loss_dyn'] = np.mean(val_utility_loss_dyn)
            losses_dict['val_utility_loss_stat'] = np.mean(val_utility_loss_stat)
    if emb_adv:
        losses_dict['priv_training_loss'] = np.mean(priv_training_loss)
        losses_dict['util_training_loss'] = np.mean(util_training_loss)
    if use_discrim_adv:
        losses_dict['discriminator_loss'] = np.mean(discriminator_loss)
        losses_dict['discriminator_acc'] = np.mean(discriminator_acc)
        if run_eval:
            losses_dict['val_discriminator_loss'] = np.mean(val_discriminator_loss)
            losses_dict['val_discriminator_acc'] = np.mean(val_discriminator_acc)
    if discrim_adv:
        losses_dict['discriminator_train_loss'] = np.mean(discriminator_train_losses)
    return losses_dict

In [58]:
training_stages = [
    {'epochs': 500, 'paired': False, 'ae': True, 'ee': True, 'cross': False, 'triplet': False, 'train_emb_adv': False, 'train_discrim_adv': False, 'emb_adv': False, 'discrim_adv': False, 'eval': True, 'sgn_eval': True, 'save': True},
    {'epochs': 100, 'paired': False, 'ae': True, 'ee': True, 'cross': False, 'triplet': False, 'train_emb_adv': True, 'train_discrim_adv': True, 'emb_adv': False, 'discrim_adv': False, 'eval': True, 'sgn_eval': True, 'save': True},
    {'epochs': 200, 'paired': False, 'ae': True, 'ee': True, 'cross': False, 'triplet': False, 'train_emb_adv': True, 'train_discrim_adv': True, 'emb_adv': True, 'discrim_adv': True, 'eval': True, 'sgn_eval': True, 'save': True},
    {'epochs': 100, 'paired': True, 'ae': True, 'ee': True, 'cross': True, 'triplet': False, 'train_emb_adv': True, 'train_discrim_adv': True, 'emb_adv': True, 'discrim_adv': True, 'eval': True, 'sgn_eval': True, 'save': True},
    {'epochs': 1, 'paired': True, 'ae': True, 'ee': False, 'cross': False, 'triplet': False, 'train_emb_adv': False, 'train_discrim_adv': False, 'emb_adv': False, 'discrim_adv': False, 'eval': True, 'sgn_eval': True, 'save': True},
]
total_epochs = sum([stage['epochs'] for stage in training_stages])

# mlflow logging
try: mlflow.end_run()
except: pass
mlflow.start_run()
mlflow.log_param('total_epochs', total_epochs)
mlflow.log_param('batch_size', batch_size)
mlflow.log_param('learning_rate', lr)
mlflow.log_param('one_dimension_conv', one_dimension_conv)
mlflow.log_param('ntu120', ntu_120)
mlflow.log_param('train_equal_test', str(not seperate_train_test))
mlflow.log_params(model.get_loss_params())

# os.mkdir('training_stages_log')
training_stage_name = f'training_stages_log/stages{datetime.now().strftime("%Y%m%d-%H%M%S")}.json'
with open(training_stage_name, 'w') as f:
    json.dump(training_stages, f)
mlflow.log_artifact(training_stage_name)

k=3

stages_save_path = f'pretrained/{datetime.now().strftime("%Y%m%d-%H%M%S")}'
os.mkdir(stages_save_path)

for i, stage in enumerate(training_stages):
    print('\nMoving to new stage')
    print(stage, '\n')
    for epoch in range(stage['epochs']):
        if stage['sgn_eval']:
            if validation_acc_freq > 0 and epoch % validation_acc_freq == 0: use_sgn = True
            else: use_sgn = False
        else: use_sgn = False
        if not stage['paired']:
            log_dict = train_unpaired(epoch, run_eval=stage['eval'], run_sgn_eval= use_sgn, save=stage['save'], ae=stage['ae'], ee=stage['ee'], triplet=stage['triplet'], use_emb_adv=stage['emb_adv'], use_discrim_adv=stage['discrim_adv'], emb_adv=stage['train_emb_adv'], discrim_adv=stage['train_discrim_adv'], k=k)
        else: 
            log_dict = train_paired(epoch, train_ae=stage['ae'], train_cross=stage['cross'], train_discrim=stage['train_discrim_adv'], train_emb_adv=stage['train_emb_adv'], run_eval=stage['eval'], use_emb_adv=stage['emb_adv'], use_discrim_adv=stage['discrim_adv'], run_sgn_eval= use_sgn, save=stage['save'], k=k)
        
        for key, value in log_dict.items():
            mlflow.log_metric(key, value, step=epoch)

    # save model
    torch.save(model.state_dict(), f'{stages_save_path}/stage_{i}.pt')

mlflow.end_run()


Moving to new stage
{'epochs': 500, 'paired': False, 'ae': True, 'ee': True, 'cross': False, 'triplet': False, 'train_emb_adv': False, 'train_discrim_adv': False, 'emb_adv': False, 'discrim_adv': False, 'eval': True, 'sgn_eval': True, 'save': True} 

--------------------
Epoch 1/901
--------------------
Training Loss:			0.011131992805469946
Validation Loss:		0.0094847207524958

Training Losses:
Reconstruction Loss:		0.003566723280463933
End Effector Loss:		0.003998546241256991
Triplet Loss:			0.0
Privacy Loss:			0.0
Privacy Loss Dyn:		0.0
Privacy Loss Stat:		0.0
Utility Loss:			0.0
Utility Loss Dyn:		0.0
Utility Loss Stat:		0.0
Discriminator Loss:		0.0

Validation Losses:
Val Reconstruction Loss:	0.0027581836629009637
Val End Effector Loss:		0.003968353407018044
Val Triplet Loss:		0.0
Val Privacy Loss:		0.0
Val Privacy Loss Dyn:		0.0
Val Privacy Loss Stat:		0.0
Val Utility Loss:		0.0
Val Utility Loss Dyn:		0.0
Val Utility Loss Stat:		0.0
Val Discriminator Loss:		0.0



Known Action Ac

## Visualize Results

In [ ]:
render_video(val_data[0][0][2][:, :, :3])

In [ ]:
render_video(model(val_data[0][0][2][:, :, :3].unsqueeze(0).to(device), val_data[0][0][2][:, :, 3:].unsqueeze(0).to(device)).squeeze(0).cpu().detach().numpy())

# Retargeting

In [ ]:
def retarget():
    X_hat_random = {}
    X_hat_constant = {}

    x2_const = X[random.sample(list(X.keys()), 1)[0]].float().cuda().unsqueeze(0)
    with torch.no_grad():
        for file in X:
            x1 = X[file].unsqueeze(0)
            x2_random = X[random.sample(list(X.keys()), 1)[0]].unsqueeze(0)
            X_hat_random[file] = model.eval(x1.float().cuda(), x2_random.float().cuda()).cpu().numpy().squeeze()
            X_hat_constant[file] = model.eval(x1.float().cuda(), x2_const).cpu().numpy().squeeze()

    # Save results
    with open('results/X_hat_random.pkl', 'wb') as f:
        pickle.dump(X_hat_random, f)
    with open('results/X_hat_constant.pkl', 'wb') as f:
        pickle.dump(X_hat_constant, f)

# retarget()

# Evaluate utility of other baselines

In [ ]:
def anonymizer_to_sgn(t, max_frames=300):
    xyz, frames, joints, actors = t.shape
    
    # Pre-allocate memory for the output array
    X = np.zeros((max_frames, xyz * joints * actors), dtype=np.float32)
    
    # Reshape the input array for easier manipulation
    t_reshaped = t.reshape((frames, -1))
    
    # Copy over the reshaped data to the pre-allocated output
    X[:frames, :t_reshaped.shape[1]] = t_reshaped
    
    return X

In [ ]:
# Classical MR
x_pkl = 'C:\\Users\\Carrt\\OneDrive\\Code\\Motion Privacy\\Defense Models\\Mean Skeleton\\X_FileNameKey_SingleActor_filtered.pkl'
from_moon = False
pad_data = False
# trim the data to 75 joints

# DMR
# x_pkl = 'C:\\Users\\Carrt\\OneDrive\\Code\\Motion Retargeting\\results\\X_hat_constant.pkl'
# from_moon = False
# pad_data = True

# Moon ResNet
# x_pkl = 'C:\\Users\\Carrt\\OneDrive\\Code\\Motion Privacy\\External Repositories\\Skeleton-anonymization\\X_resnet_file.pkl'
# from_moon = True
# pad_data = False

# Moon UNet
# x_pkl = 'C:\\Users\\Carrt\\OneDrive\\Code\\Motion Privacy\\External Repositories\\Skeleton-anonymization\\X_unet_file.pkl'
# from_moon = True
# pad_data = False

with open(x_pkl, 'rb') as f:
    test_x = pickle.load(f)

if from_moon:
    test_x = {k: v[0] for k, v in test_x.items()}

    for file in test_x:
        test_x[file] = anonymizer_to_sgn(test_x[file])[:, :75]

if pad_data:
    # pad data to 300 frames
    for file in test_x:
        if test_x[file].shape[0] == 1:
            test_x[file] = test_x[file][0]
        test_x[file] = np.pad(test_x[file], ((0, 300-test_x[file].shape[0]), (0, 0)), 'constant')

In [ ]:
sgn_train_x, sgn_train_y, sgn_val_x, sgn_val_y = np.zeros((batch_size, 300, 150)), np.zeros((batch_size, 1)), np.zeros((batch_size, 300, 150)), np.zeros((batch_size, 1))

def eval(X_dict):
    # Remove NTU120 if needed
    if not ntu_120:
        X_dict = {k: v for k, v in X_dict.items() if int(k[17:20]) <= 60}

    # Convert to tensor
    x = np.zeros((len(X_dict), 300, 150), dtype=np.float32)
    y_util = np.zeros(len(X_dict))
    y_priv = np.zeros(len(X_dict))

    for i, file in enumerate(X_dict):
        if X_dict[file].shape[1] == 75:
            X_dict[file] = np.pad(X_dict[file], ((0, 0), (0, 75)), 'constant')

        x[i] = np.array(X_dict[file], dtype=np.float32)
        y_util[i] = int(file[17:20])
        y_priv[i] = int(file[9:12])

    y_util = y_util - 1
    y_priv = y_priv - 1
    y_util = np.eye(utility_classes)[y_util.astype(int)]
    y_priv = np.eye(privacy_classes)[y_priv.astype(int)]

    acc, f1, prec, recall, topk = run_sgn_eval(sgn_train_x, sgn_train_y, x, y_util, sgn_val_x, sgn_val_y, 1, sgn_ar, k=3)
    print(f'Utility Accuracy:\t\t{acc}\nUtility F1:\t\t\t{f1*100}\nUtility Precision:\t\t{prec*100}\nUtility Recall:\t\t\t{recall*100}\n')

    acc, f1, prec, recall, topk = run_sgn_eval(sgn_train_x, sgn_train_y, x, y_priv, sgn_val_x, sgn_val_y, 1, sgn_priv, k=3)
    print(f'Privacy Accuracy:\t\t{acc}\nPrivacy F1:\t\t\t{f1*100}\nPrivacy Precision:\t\t{prec*100}\nPrivacy Recall:\t\t\t{recall*100}\n')

# eval(cmr)
# eval(resnet)
# eval(test_x)